In [2]:
# Bike Sharing Demand Kaggle project

# W207 Final Project

# Chris Murray, Rahul Ragunathan,  Mahadevan Rajagopalan

#
# Import the needed libraries.
import csv
import re
import datetime
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [43]:
# setup the training and test data
def read_data(filename, is_train):

    with open(filename, 'rb') as csvfile:
       # skip the header
       header_line = next(csvfile)
       csvreader = csv.reader(csvfile, delimiter=',')
       # Initialize some empty lists
       time       = []
       season     = []
       holiday    = []
       workingday = []
       weather    = []
       temp       = []
       atemp      = []
       humidity   = []
       windspeed  = []
       # There are three additional fields with training data
       if is_train:
         casual     = []
         registered = []
         count      = []
       # Each row is a collection of strings
       for row in csvreader:
          time.append(row[0])
          season.append((int)(row[1]))
          holiday.append((int)(row[2]))
          workingday.append((int)(row[3]))
          weather.append((int)(row[4]))
        
          # Convert the float values to int values
          temp.append((int)(float(row[5])))
          atemp.append((int)(float(row[6])))
          humidity.append(int(float(row[7])))
          windspeed.append(int(float(row[8])))
          if is_train:
            casual.append(int(row[9]))
            registered.append(int(row[10]))
            count.append(int(row[11]))
            
       # The time is in year month day hour min secs  format.
       # We just need the time of the day. This will be converted into the number of seconds
       # since the start of the day.
       
       splittime = []
       for str in time:
         splittime.append( str.split(" ")[1])
        
       # convert to number of seconds
       secondstime=[]
       for str in splittime:
         h,m,s = re.split(':',str)
         secondstime.append("%d" % (int)(datetime.timedelta(hours=int(h),minutes=int(m),seconds=int(s)).total_seconds()))
       
       num_features = 8
       
       data = np.zeros((len(season), num_features), dtype=np.int)
    
       # copy the the feature values to the data array
       for i in range(data.shape[0]):
          data[i][0] = secondstime[i]
          data[i][1] = season[i]
          data[i][2] = workingday[i]
          data[i][3] = weather[i]
          data[i][4] = temp[i]
          data[i][5] = atemp[i]
          data[i][6] = humidity[i]
          data[i][7] = windspeed[i]
        
    
       if is_train:
          return data, count
       else:
          return data, time
            
# This function writes a row to a csv file.    
def write_csv(filename, row):
    b = open(filename, 'w')
    writer = csv.writer(b)
    
    writer.writerows(row)
    #for i in range(len(preds)+1):
    #  writer.writerow(row[i])
      
    b.close()
    return

# Read the training data   
train_data, train_labels = read_data('train.csv', True) 

# Read the test data 
test_data, time = read_data('test.csv', False)

# Instantiate the classifier
clf = KNeighborsClassifier(n_neighbors=5)
    
# Fit the training data
clf.fit(train_data, train_labels)

# Make predictions
preds= clf.predict(test_data)

l= ["datetime", "count"]
for i in range (len(preds)):
    l.append(time[i]) 
    l.append("%d" %preds[i])
            
pred_output = np.array(l).reshape((len(preds)+1, 2))

# Write the predictions to a csv file
write_csv('predict1.csv', pred_output)
      

      
    
    

        
    


    

    
    
                          
        
                          
    
        
    
    
    
        
        
    
    